# Book Scraper Visualization

This notebook provides an interactive UI to explore the scraped book data.

In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import json

# Set pandas display options
pd.set_option('display.max_rows', 15)
pd.set_option('display.max_colwidth', None)

In [2]:
# Load Data
try:
    with open('output/books.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    df = pd.DataFrame(data)
    print(f"Loaded {len(df)} books.")
except FileNotFoundError:
    print("Error: output/books.json not found. Please run the scraper first.")
    df = pd.DataFrame()

Loaded 1000 books.


In [ ]:
if not df.empty:
    # Preprocessing
    # Convert PRICE to float
    df['price_numeric'] = df['price'].str.replace('£', '', regex=False).astype(float)
    
    # Convert RATING to numeric
    rating_map = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
    df['rating_num'] = df['rating'].map(rating_map)
    
    # Widgets
    style = {'description_width': 'initial'}
    
    title_search = widgets.Text(
        description='🔍 Search Title:', 
        placeholder='Type to search...',
        style=style, 
        layout=widgets.Layout(width='300px')
    )
    
    rating_filter = widgets.Dropdown(
        options=['All', '⭐', '⭐⭐', '⭐⭐⭐', '⭐⭐⭐⭐', '⭐⭐⭐⭐⭐'],
        value='All',
        description='Rating:',
        style=style,
        layout=widgets.Layout(width='150px')
    )

    rating_mode = widgets.Checkbox(
        value=False,
        description='Exact Match',
        indent=False,
        layout=widgets.Layout(width='100px')
    )
    
    price_min = df['price_numeric'].min()
    price_max = df['price_numeric'].max()
    
    price_slider = widgets.FloatRangeSlider(
        value=[price_min, price_max],
        min=price_min,
        max=price_max,
        step=0.5,
        description='💰 Price Range (£):',
        style=style,
        layout=widgets.Layout(width='400px')
    )

    reset_button = widgets.Button(
        description='Reset Filters',
        icon='refresh',
        button_style='info',
        layout=widgets.Layout(width='120px')
    )
    
    # Display Logic accoordingly
    def render_books(title, rating, exact_rating, price_range):
        filtered = df.copy()
        
        # Filter Title
        if title:
            filtered = filtered[filtered['title'].str.contains(title, case=False, na=False)]
        
        # Filter Rating
        if rating != 'All':
            stars = len(rating)
            if exact_rating:
                filtered = filtered[filtered['rating_num'] == stars]
            else:
                filtered = filtered[filtered['rating_num'] >= stars]
            
        # Filter Price
        filtered = filtered[(filtered['price_numeric'] >= price_range[0]) & (filtered['price_numeric'] <= price_range[1])]
        
        # Display Status
        display(widgets.HTML(f"<b>Showing {len(filtered)} books</b>"))
        
        if filtered.empty:
            print("No books found matching criteria.")
            return
            
        # Eye-Catching Table Styling
        styled_table = filtered[['title', 'price', 'rating', 'availability', 'page']].style \
            .background_gradient(subset=['page'], cmap='Blues') \
            .set_properties(**{'text-align': 'left', 'font-size': '11pt', 'border-color': 'lightgray'}) \
            .set_table_styles([{'selector': 'th', 'props': [('background-color', '#4CAF50'), ('color', 'white'), ('font-weight', 'bold'), ('text-align', 'left')]}])
            
        display(styled_table)

    # Callback for Reset Button
    def reset_filters(b):
        title_search.value = ''
        rating_filter.value = 'All'
        rating_mode.value = False
        price_slider.value = [price_min, price_max]

    reset_button.on_click(reset_filters)

    # Layout
    ui = widgets.VBox([
        widgets.HBox([title_search, rating_filter, rating_mode]),
        widgets.HBox([price_slider, reset_button])
    ], layout=widgets.Layout(padding='20px', border='1px solid #ddd', margin='0 0 20px 0'))
    
    display(widgets.HTML("<h2>📚 Book Browser</h2>"))
    output = widgets.interactive_output(render_books, {
        'title': title_search, 
        'rating': rating_filter, 
        'exact_rating': rating_mode,
        'price_range': price_slider
    })
    
    display(ui, output)
else:
    print("No data to display.")

HTML(value='<h2>📚 Book Browser</h2>')

Output()